# Data separation

## Compressed sensing 

Mohamed BOUSSENA and Ngoc-Tam Le

This project is based on the following article : 
Data Separation by Sparse Representations, Gitta Kutyniok, 2011

https://arxiv.org/pdf/1102.4527.pdf

In [ ]:
import numpy as np
from cvxopt import solvers, matrix

## Theoretical results
___

The data separation problem is set as follow : Given a vector $x \in \mathcal{H}$ (where $\mathcal{H}$ is assumed to be a Hilbert space) such that $x = x_1 + x_2$, we would like to extract the two components $x_1$ and $x_2$. A general idea underlying data separation is to choose two basis representation $\Phi_1$ and $\Phi_2$ such that $x_1$ and $x_2$ can be represented in a sparse way in one of the two basis basis. Precisely, we would like to choose the basis such that $x_1$ is sparse in $\Phi_1$ and not $\Phi_2$, and vice versa. 

__Definition__ (mutual coherence) : Given a frame $\Phi$ on a Hilbert space $\mathcal{H}$, we define the mutual coherence for $\Phi$ by :

\begin{equation*}
    \mu(\Phi) = \underset{i,j \in I, i \neq j}{max}  \left| \langle \phi_i, \phi_j \rangle \right|
\end{equation*}
___

The following result gives us the guarantee that solving $l_1$ minimisation is equivalent to the $l_0$ minimisation.

__Theorem__: Let $\Phi_1$ and $\Phi_2$ two frames on a Hilbert space $\mathcal{H}$, and $x \in \mathcal{H} \backslash \{0\}$. Then if $x = \left[\Phi_1 \middle| \Phi_2 \right] c$ and 
    
$$\|c \|_0 \leq \frac{1}{2} \left(1 + \frac{1}{\mu\left(\left[\Phi_1 \middle| \Phi_2 \right]\right)} \right)$$

Then the solution of the $l_1$ minimization problem coincide with the solution of the $l_0$ minimization problem.
___

We want to solve the following program :


$$\underset{c_1, c_2}{min}\hspace{0.3cm} \|c_1 \|_1 + \|c_2\|_1$$
$$s.t. x = \left[\Phi_1 \middle|  \Phi_2 \right] \begin{bmatrix}c_1 \\ c_2 \end{bmatrix} $$

The constraint can be rewritten 

We can rewrite it as a linear program

$$min \ \mathbb{1}^T ( c_1^{+} + c_2^{+} + c_1^{-} + c_2^{-}) $$

$$ s.t \hspace{1cm} 
\begin{aligned}
\begin{bmatrix} \Phi_1 & \Phi_2 & -\Phi_1 & -\Phi_2 \end{bmatrix} \begin{bmatrix} c_1\\ c_2 \\ c_1^{-} \\ c_2^{-} \end{bmatrix} = x\\
0 \leq c_i^{+}, 0 \leq c_i^{-}, i=1, 2
\end{aligned}$$

In [10]:
class DataSep():
    
    def __init__(self, basis1, basis2, x):
        self.basis1 = matrix(basis1) # ndarray of size n * d1
        self.basis2 = matrix(basis2) # ndarray of size n * d2
        self.x = matrix(x) # 1d array of size n 
        
    def coherence(self, phi):
        """
        compute the mutual coherence on a frame phi
        """
        mu = np.max( np.dot(phi, phi.T))
        return mu
    
    def get_sparse_component(self):
        """
        solve the basis pursuit program
        """
        A = matrix(np.hstack((self.basis1, self.basis2, -self.basis1, -self.basis2)).T)
        b = self.x
        dim = A.shape[0]
        
        c = matrix( [1] * dim )
        G = -matrix(np.eye(dim))
        h = matrix([0] * dim)
        sol = cvxopt.solvers.lp(c, G, h, A, b)
        
        return sol


In [6]:
# to complete
basis1 = None
basis2 = None
data_sep = DataSep(basis1, basis2, x)

## Application